In [1]:
# Load AIEEE Data

In [1]:
%cd ../../

/home/souvic/mounted/btp/vahini/Name2Demographics


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 


import sys  
sys.path.insert(0, 'Models/AIEEEGenderData/Preprocessing/')
from aieee_gender import *

In [69]:
aieee_df = AIEEEGenderData()

Load Models

In [10]:
#  other models
AIEEE_BERT = "Models/AIEEEGenderData/BERT/SavedModel/BERT"
CBSE_BERT = "Models/CBSEData/BERT/SavedModel/BERT"
ER_BERT = "Models/ERData/BERT/SavedModel/BERT"

In [5]:
%%capture
!pip install transformers[sentencepiece]
import transformers

In [32]:
from transformers import BertModel, AutoModelForSequenceClassification
from transformers import AutoTokenizer

In [38]:
aieee_model = AutoModelForSequenceClassification.from_pretrained(AIEEE_BERT)

In [39]:
cbse_model = AutoModelForSequenceClassification.from_pretrained(CBSE_BERT)

In [33]:
er_model = AutoModelForSequenceClassification.from_pretrained(ER_BERT)

In [27]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Predict Gender

In [25]:
from transformers import pipeline

In [40]:
pipe = pipeline(task="text-classification",model=aieee_model, tokenizer=tokenizer)

In [73]:
#  0 female and 1 male 
predictions = pipe(list(aieee_df['Name']), batch_size=20)

RuntimeError: [enforce fail at CPUAllocator.cpp:67] . DefaultCPUAllocator: can't allocate memory: you tried to allocate 21374552064 bytes. Error code 12 (Cannot allocate memory)

In [63]:
labels = []
for pred in predictions:
    if pred['label'] == 'LABEL_0':
        labels.append(0)
    else:
        labels.append(1)

In [66]:
true = list(aieee_df[:100]['Gender'])

In [67]:
from sklearn.metrics import classification_report

print(classification_report(true, labels, digits = 4))

              precision    recall  f1-score   support

         0.0     1.0000    0.9300    0.9637       100
         1.0     0.0000    0.0000    0.0000         0

    accuracy                         0.9300       100
   macro avg     0.5000    0.4650    0.4819       100
weighted avg     1.0000    0.9300    0.9637       100



/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/souvic/.conda/envs/py38/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
import math

def print_results(rmodel, tokenizer, df):
    pipe = pipeline(task="text-classification",model=rmodel, tokenizer=tokenizer)
    n = df.shape[0]
    predictions = []
    for i in range(math.ceil(n/1000)):
        df1 = df[i*1000:(i+1)*(1000)]
        onway = pipe(list(df1['Name']), batch_size=256)
        predictions = predictions + onway
#     import pdb; pdb.set_trace()
    labels = []
    for pred in predictions:
        if pred['label'] == 'LABEL_0':
            labels.append(0)
        else:
            labels.append(1)
    
    true = list(df['Gender'])
    print(classification_report(true, labels, digits = 4))
    return 

In [86]:
print_results(aieee_model, tokenizer, aieee_df)

ValueError: Found input variables with inconsistent numbers of samples: [409286, 409000]

In [91]:
print_results(cbse_model, tokenizer, aieee_df)

              precision    recall  f1-score   support

         0.0     0.8646    0.9051    0.8844    175931
         1.0     0.9258    0.8932    0.9092    233355

    accuracy                         0.8983    409286
   macro avg     0.8952    0.8991    0.8968    409286
weighted avg     0.8995    0.8983    0.8985    409286



In [ ]:
print_results(er_model, tokenizer, aieee_df)